In [1]:
import torch, torch.autograd as autograd
import torch.nn as nn, torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable as avar
    
from SimpleTask import SimpleGridTask
from TransportTask import TransportTask
from NavTask import NavigationTask
from SeqData import SeqData
from LSTMFM import LSTMForwardModel

import os, sys, pickle, numpy as np, numpy.random as npr, random as r

In [2]:
f_model_name = 'LSTM_FM_1_99'    
s = 'navigation' # 'transport'
trainf, validf = s + "-data-train-small.pickle", s + "-data-test-small.pickle"
print('Reading Data')
train, valid = SeqData(trainf), SeqData(validf)

Reading Data
Reading navigation-data-train-small.pickle
	Built
Reading navigation-data-test-small.pickle
	Built


In [3]:
fm = LSTMForwardModel(train.lenOfInput,train.lenOfState)
fm.load_state_dict( torch.load(f_model_name) )

In [4]:
class HenaffPlanner():

    def __init__(self,forward_model,env, maxNumActions=1,noiseSigma=0.0,startNoiseSigma=0.1,niters=200):
        # Parameters
        self.sigma = noiseSigma
        self.start_sigma = startNoiseSigma
        self.nacts = maxNumActions
        self.niters = niters
        # The forward model
        self.f = forward_model
        # Stop forward model from training
        for p in self.f.parameters(): p.requires_grad = False
        # Get shapes from forward model
        self.state_size = self.f.stateSize
        self.action_size = self.f.inputSize - self.f.stateSize
        self.env = env

    def generatePlan(self,
            start_state,         # The starting state of the agent
            eta=0.0003,            # The learning rate given to ADAM
            noiseSigma=None,     # Noise strength on inputs. Overwrites the default setting from the init
            niters=None,         # Number of optimization iterations. Overwrites the default setting from the init
            goal_state=None,     # Use to specify a goal state manually (instead of reading it from a given state)
            useCE=False,         # Specifies use of the cross-entropy loss, taken over subvectors of the state
            verbose=False,       # Specifies verbosity
            extraVerbose=False,  # Specifies extra verbosity
            useGumbel=True,      # Whether to use Gumbel-Softmax in the action sampling
            temp=0.01,           # The temperature of the Gumbel-Softmax method
            lambda_h=0.0,        # Specify the strength of entropy regularization (negative values encourage entropy)
            useIntDistance=False
        ):

        # Other settings
        useIntDistance = False # Note: does not apply if using CE
        useMSE_loss = (not useIntDistance) and (not useCE)
        if not noiseSigma is None: self.sigma = noiseSigma
        if not niters is None: self.niters = niters

        # Initialize random actions and optimizer
        x_t = avar( torch.randn(self.nacts, self.action_size) * self.start_sigma, requires_grad=True )
        optimizer = torch.optim.Adam( [x_t], lr=eta )
        # Choose loss function
        if useCE:
            lossf = nn.CrossEntropyLoss()
            if verbose: print('Using CE loss')
        elif useMSE_loss:
            if verbose: print('Using MSE loss')
            lossf = nn.MSELoss()
        else:
            if verbose: print('Using int distance loss')

        # Set goal state
        deconStartState = self.env.deconcatenateOneHotStateVector(start_state)
        if goal_state is None:
            gx, gy = avar(torch.FloatTensor(deconStartState[-2])), avar(torch.FloatTensor(deconStartState[-1]))
        else: print('Not yet implemented'); sys.exit(0)

        # Indices of start state position
        sindx = avar(torch.FloatTensor(deconStartState[0])).max(0)[1]
        sindy = avar(torch.FloatTensor(deconStartState[1])).max(0)[1]

        # Indices of goal state position
        indx, indy = gx.max(0)[1], gy.max(0)[1]

        # Start optimization loop
        for i in range(self.niters):
            # Generate soft action sequence
            epsilon = avar( torch.randn(self.nacts, self.action_size) * self.sigma )
            # Add noise to current action sequence
            y_t = x_t + epsilon
            # Softmax inputs to get current soft actions
            a_t = F.softmax( y_t, dim=1 )
            # Compute predicted state
            #currState = avar(torch.FloatTensor(start_state)).unsqueeze(0)
            # Loop over actions to obtain predicted state
            self.f.reInitialize(1)
            currState, intStates = self.f.forward(start_state, a_t, self.nacts)
            # Now have final (predicted) result of action sequence
            # Extract predicted position of current state
            pvx = currState[0:15] 
            pvy = currState[15:30] 
            # Compute loss
            if useCE: # Cross-entropy loss
                lossx = lossf(pvx.view(1,pvx.shape[0]), indx) 
                lossy = lossf(pvy.view(1,pvy.shape[0]), indy)
            elif useIntDistance: # Integer distance loss
                ints = avar( torch.FloatTensor( list(range(15)) ) )
                prx = torch.sum( ints * pvx )
                pry = torch.sum( ints * pvy )
                lossx = (1.0/15.0) * (prx - indx.data[0]).pow(2)
                lossy = (1.0/15.0) * (pry - indy.data[0]).pow(2) 
            else: # Using MSE loss via one-hot pos
                lossx = lossf(pvx, gx)
                lossy = lossf(pvy, gy)
            # Entropy penalty
            H = -torch.sum( torch.sum( a_t*torch.log(a_t) , dim = 1 ) )
            # Final loss function 
            loss = lossx + lossy + lambda_h * H
            # Print status
            if extraVerbose:
                a_inds = ",".join([ str(a.max(dim=0)[1].data[0]) for a in a_t  ]) 
                print(i,'->','Lx =',lossx.data[0],', Ly =',lossy.data[0],', H =',H.data[0],', TL =',loss.data[0],', A =',a_inds)
            # Clear the optimizer gradient
            optimizer.zero_grad()
            # Back-prop the errors to get the new gradients
            loss.backward(retain_graph=True)
            optimizer.step()
            # Print the predicted result at the current iteration
            if extraVerbose:
                print('Predicted End:',pvx.max(0)[1].data[0],pvy.max(0)[1].data[0])
            # Ensure the x_t gradients are cleared
            #x_t.grad.data.zero_()
        # Print and analyze the final plan, if desired
        if verbose:
            print('\nEnd\n')
            print('Actions')
            for k in range(0,self.nacts):
                action = F.softmax( x_t[k,:], dim=0 )
                print(action.max(0)[1].data[0],end=' -> ')
                print(NavigationTask.actions[action.max(0)[1].data[0]],end=' ')
                print(action.data)
            print('--')
            print('START ',sindx.data[0],sindy.data[0])
            print('TARGET END ',indx.data[0],indy.data[0])
            print('PREDICTED END',pvx.max(0)[1].data[0], pvy.max(0)[1].data[0])
            print('--')
        # Return the final action sequence 
        return [ x.max(0)[1].data[0] for x in x_t ]


In [5]:
start = np.zeros(64)
start[0] = 1
start[15] = 1
start[15+15] = 1
start[15+15+4+5] = 1
start[15+15+4+15+8] = 1
env = NavigationTask()
#print(fm.env.deconcatenateOneHotStateVector(start))
print('Building planner')
planner = HenaffPlanner(fm,env,maxNumActions=6)
print('Starting generation')
actions = planner.generatePlan(
                    start,
                    eta=0.3,
                    noiseSigma=1.0,
                    niters=300,
                    goal_state=None,
                    useCE=True,
                    verbose=True,
                    extraVerbose=False,
                    useGumbel=True,
                    temp=0.1,
                    lambda_h=-0.005,
                    useIntDistance=False
                    )
print('FINAL ACTIONS:', actions)


Building planner
Starting generation
Using CE loss

End

Actions
8 -> Move_4 
 8.3581e-05
 4.5191e-04
 2.0505e-05
 2.4500e-05
 7.8348e-06
 3.0349e-04
 4.4709e-03
 1.9184e-02
 9.7418e-01
 1.2683e-03
[torch.FloatTensor of size 10]

7 -> Move_3 
 2.6539e-06
 4.5654e-06
 2.7681e-06
 3.8603e-07
 1.0370e-06
 1.7195e-02
 6.7890e-05
 9.8235e-01
 1.9338e-04
 1.8567e-04
[torch.FloatTensor of size 10]

5 -> Move_1 
 0.0046
 0.0003
 0.0000
 0.0001
 0.0052
 0.9892
 0.0001
 0.0003
 0.0001
 0.0001
[torch.FloatTensor of size 10]

2 -> Face_east 
 5.1321e-06
 5.9366e-05
 9.9977e-01
 3.3542e-06
 1.1887e-04
 8.4190e-06
 1.0562e-05
 9.3773e-06
 1.1992e-05
 2.3158e-06
[torch.FloatTensor of size 10]

2 -> Face_east 
 7.4292e-06
 5.8189e-06
 9.9992e-01
 2.3299e-06
 1.8662e-05
 1.0771e-05
 4.7657e-06
 9.9611e-06
 8.2497e-06
 7.0630e-06
[torch.FloatTensor of size 10]

9 -> Move_5 
 4.8282e-06
 3.5284e-06
 8.1347e-06
 4.9042e-06
 5.0334e-06
 2.5757e-06
 2.2829e-06
 1.6245e-05
 1.5745e-05
 9.9994e-01
[torch.Floa

In [6]:
##################### Hyper-params #####################
# lambda_hs = [0.0,0.01,-0.01,0.05,-0.05,0.005,-0.005]            # Entropy strength
# etas = [0.5,0.25,0.1,0.05,0.025,0.01,0.005,0.001,0.0005]        # Learning rate
# useGumbels = [True,False]                                       # Whether to use Gumbel-softmax
# temperatures = [0.1,0.01,0.001,1.0]                             # Temperature for Gumbel-softmax
# noiseSigmas = [0.0,0.01,0.02,0.05,0.1,0.25,0.5,0.75,1.0,1.25]   # Noise strength on input
## Init try
# lambda_hs = [0.0,0.005,-0.005]                                  # Entropy strength
# etas = [0.5,0.25,0.1,0.05,0.025,0.01,0.005,0.001,0.0005]        # Learning rate
# useGumbels = [True,False]                                       # Whether to use Gumbel-softmax
# temperatures = [0.1,0.01,0.001]                             # Temperature for Gumbel-softmax
# noiseSigmas = [0.0,0.05,0.1,0.5,1.0]   # Noise strength on input
## Only use ones with decent results
lambda_hs = [0.0,-0.005, 0.005]                                  # Entropy strength
etas = [0.2,0.3,1,0.1, 0.5]        # Learning rate
useGumbels = [True, False]                                       # Whether to use Gumbel-softmax
temperatures = [2,1, 10, 0.1,0.001]                             # Temperature for Gumbel-softmax
noiseSigmas = [0.5,0.05, 1.0, 0.75]   # Noise strength on input
########################################################

###### Settings ######
niters = 150
verbose = False
extraVerbose = False
numRepeats = 5
fileToWriteTo = 'hyper-param-results.txt' # Set to None to do no writing
distType = 1 # 0 = MSE, 1 = CE, 2 = dist
######################

# Build an env with the given INT inputs
def generateTask(px,py,orien,gx,gy):
    direction = NavigationTask.oriens[orien]
    gs = np.array([gx, gy])
    env = NavigationTask(agent_start_pos=[np.array([px,py]), direction],goal_pos=gs)
    return env

# Function for running a single suite of tests (on one hyper-param set)
def runTests(lh,eta,noiseLevel,ug,cnum,temp=None,distType=0):
    # Define tasks
    tasks = [
        [3, generateTask(0,0,0,5,5)],
        [4, generateTask(5,5,1,0,9)],
        [5, generateTask(3,2,2,7,7)],
    ]
    # Choose dist type
    if distType == 0:   useCE = False; intDist = False
    elif distType == 1: useCE = True;  intDist = False
    elif distType == 2: useCE = False; intDist = True 
    # Display status
    wstring = cnum + ',lambda_h=' + str(lh) + ',eta=' + str(eta) + ',sigma=' + str(noiseLevel) + ',dType=' + str(distType) + ',ug=' + str(ug)
    if ug: wstring += ',temp=' + str(temp) 
    # For each tasks, repeated a few times, attempt to solve the problem
    score, tot = 0, 0
    for i, task in enumerate(tasks):
        #print(i)
        for _ in range(numRepeats):
            planner = HenaffPlanner(fm,env,maxNumActions=task[0])
            cenv = task[1]
            actions = planner.generatePlan(
                    cenv.getStateRep(oneHotOutput=True),
                    eta=eta,
                    noiseSigma=noiseLevel,
                    niters=niters,
                    goal_state=None,
                    useCE=True,
                    verbose=verbose,
                    extraVerbose=extraVerbose,
                    useGumbel=ug,
                    temp=temp,
                    lambda_h=lh,
                    useIntDistance=intDist )
            # Check for correctness
            for a in actions: cenv.performAction( a )
            r = cenv.getReward()
            correct = (r==1)
            tot += 1
            if correct: score += 1
    wstring += ' -> Score:' + str(score) + '/' + str(tot)
    print(wstring)
    # Write output
    if not fileToWriteTo is None:
        with open(fileToWriteTo,'a') as filehandle:
            filehandle.write( wstring + '\n' )

# Run tasks over all hyper-parameter settings
N_p, cp = len(lambda_hs)*len(etas)*len(noiseSigmas)*(1 + len(temperatures)), 1
for lambda_h in lambda_hs:
    for eta in etas:
        for noiseLevel in noiseSigmas:
            for ug in useGumbels:
                if ug:
                    for temp in temperatures: 
                        ps = str(cp) + '/' + str(N_p)
                        runTests(lambda_h,eta,noiseLevel,ug,ps,temp,distType=distType)
                        cp += 1
                else: 
                    ps = str(cp) + '/' + str(N_p)
                    runTests(lambda_h,eta,noiseLevel,ug,ps,distType=distType)
                    cp += 1

1/360,lambda_h=0.0,eta=0.2,sigma=0.5,dType=1,ug=True,temp=2 -> Score:5/15
2/360,lambda_h=0.0,eta=0.2,sigma=0.5,dType=1,ug=True,temp=1 -> Score:10/15
3/360,lambda_h=0.0,eta=0.2,sigma=0.5,dType=1,ug=True,temp=10 -> Score:9/15
4/360,lambda_h=0.0,eta=0.2,sigma=0.5,dType=1,ug=True,temp=0.1 -> Score:8/15
5/360,lambda_h=0.0,eta=0.2,sigma=0.5,dType=1,ug=True,temp=0.001 -> Score:8/15
6/360,lambda_h=0.0,eta=0.2,sigma=0.5,dType=1,ug=False -> Score:9/15
7/360,lambda_h=0.0,eta=0.2,sigma=0.05,dType=1,ug=True,temp=2 -> Score:5/15
8/360,lambda_h=0.0,eta=0.2,sigma=0.05,dType=1,ug=True,temp=1 -> Score:5/15
9/360,lambda_h=0.0,eta=0.2,sigma=0.05,dType=1,ug=True,temp=10 -> Score:8/15
10/360,lambda_h=0.0,eta=0.2,sigma=0.05,dType=1,ug=True,temp=0.1 -> Score:5/15
11/360,lambda_h=0.0,eta=0.2,sigma=0.05,dType=1,ug=True,temp=0.001 -> Score:5/15
12/360,lambda_h=0.0,eta=0.2,sigma=0.05,dType=1,ug=False -> Score:5/15
13/360,lambda_h=0.0,eta=0.2,sigma=1.0,dType=1,ug=True,temp=2 -> Score:9/15
14/360,lambda_h=0.0,eta=0

110/360,lambda_h=0.0,eta=0.5,sigma=1.0,dType=1,ug=True,temp=1 -> Score:13/15
111/360,lambda_h=0.0,eta=0.5,sigma=1.0,dType=1,ug=True,temp=10 -> Score:11/15
112/360,lambda_h=0.0,eta=0.5,sigma=1.0,dType=1,ug=True,temp=0.1 -> Score:10/15
113/360,lambda_h=0.0,eta=0.5,sigma=1.0,dType=1,ug=True,temp=0.001 -> Score:11/15
114/360,lambda_h=0.0,eta=0.5,sigma=1.0,dType=1,ug=False -> Score:11/15
115/360,lambda_h=0.0,eta=0.5,sigma=0.75,dType=1,ug=True,temp=2 -> Score:5/15
116/360,lambda_h=0.0,eta=0.5,sigma=0.75,dType=1,ug=True,temp=1 -> Score:9/15
117/360,lambda_h=0.0,eta=0.5,sigma=0.75,dType=1,ug=True,temp=10 -> Score:7/15
118/360,lambda_h=0.0,eta=0.5,sigma=0.75,dType=1,ug=True,temp=0.1 -> Score:9/15
119/360,lambda_h=0.0,eta=0.5,sigma=0.75,dType=1,ug=True,temp=0.001 -> Score:8/15
120/360,lambda_h=0.0,eta=0.5,sigma=0.75,dType=1,ug=False -> Score:10/15
121/360,lambda_h=-0.005,eta=0.2,sigma=0.5,dType=1,ug=True,temp=2 -> Score:5/15
122/360,lambda_h=-0.005,eta=0.2,sigma=0.5,dType=1,ug=True,temp=1 -> Sco

214/360,lambda_h=-0.005,eta=0.1,sigma=0.75,dType=1,ug=True,temp=0.1 -> Score:10/15
215/360,lambda_h=-0.005,eta=0.1,sigma=0.75,dType=1,ug=True,temp=0.001 -> Score:5/15
216/360,lambda_h=-0.005,eta=0.1,sigma=0.75,dType=1,ug=False -> Score:5/15
217/360,lambda_h=-0.005,eta=0.5,sigma=0.5,dType=1,ug=True,temp=2 -> Score:9/15
218/360,lambda_h=-0.005,eta=0.5,sigma=0.5,dType=1,ug=True,temp=1 -> Score:10/15
219/360,lambda_h=-0.005,eta=0.5,sigma=0.5,dType=1,ug=True,temp=10 -> Score:9/15
220/360,lambda_h=-0.005,eta=0.5,sigma=0.5,dType=1,ug=True,temp=0.1 -> Score:7/15
221/360,lambda_h=-0.005,eta=0.5,sigma=0.5,dType=1,ug=True,temp=0.001 -> Score:7/15
222/360,lambda_h=-0.005,eta=0.5,sigma=0.5,dType=1,ug=False -> Score:9/15
223/360,lambda_h=-0.005,eta=0.5,sigma=0.05,dType=1,ug=True,temp=2 -> Score:7/15
224/360,lambda_h=-0.005,eta=0.5,sigma=0.05,dType=1,ug=True,temp=1 -> Score:8/15
225/360,lambda_h=-0.005,eta=0.5,sigma=0.05,dType=1,ug=True,temp=10 -> Score:11/15
226/360,lambda_h=-0.005,eta=0.5,sigma=0.0

318/360,lambda_h=0.005,eta=0.1,sigma=0.5,dType=1,ug=False -> Score:8/15
319/360,lambda_h=0.005,eta=0.1,sigma=0.05,dType=1,ug=True,temp=2 -> Score:9/15
320/360,lambda_h=0.005,eta=0.1,sigma=0.05,dType=1,ug=True,temp=1 -> Score:9/15
321/360,lambda_h=0.005,eta=0.1,sigma=0.05,dType=1,ug=True,temp=10 -> Score:9/15
322/360,lambda_h=0.005,eta=0.1,sigma=0.05,dType=1,ug=True,temp=0.1 -> Score:9/15
323/360,lambda_h=0.005,eta=0.1,sigma=0.05,dType=1,ug=True,temp=0.001 -> Score:8/15
324/360,lambda_h=0.005,eta=0.1,sigma=0.05,dType=1,ug=False -> Score:7/15
325/360,lambda_h=0.005,eta=0.1,sigma=1.0,dType=1,ug=True,temp=2 -> Score:13/15
326/360,lambda_h=0.005,eta=0.1,sigma=1.0,dType=1,ug=True,temp=1 -> Score:13/15
327/360,lambda_h=0.005,eta=0.1,sigma=1.0,dType=1,ug=True,temp=10 -> Score:10/15
328/360,lambda_h=0.005,eta=0.1,sigma=1.0,dType=1,ug=True,temp=0.1 -> Score:8/15
329/360,lambda_h=0.005,eta=0.1,sigma=1.0,dType=1,ug=True,temp=0.001 -> Score:9/15
330/360,lambda_h=0.005,eta=0.1,sigma=1.0,dType=1,ug=F